# Chapter 03-02: 옵티마이저 (Optimizers)

## 학습 목표
- SGD, Adam, RMSprop, AdamW의 동작 원리를 수식으로 이해한다
- 학습률(Learning Rate)의 중요성과 적절한 설정 방법을 안다
- Learning Rate Scheduler를 활용하여 동적으로 학습률을 조정할 수 있다
- 동일한 모델에 여러 옵티마이저를 적용하고 수렴 속도를 비교할 수 있다

## 목차
1. [수학적 기초](#1.-수학적-기초)
2. [옵티마이저 생성 및 기본 사용법](#2.-옵티마이저-생성-및-기본-사용법)
3. [학습률의 영향](#3.-학습률의-영향)
4. [Learning Rate Scheduler](#4.-Learning-Rate-Scheduler)
5. [ReduceLROnPlateau 콜백](#5.-ReduceLROnPlateau-콜백)
6. [옵티마이저 수렴 속도 비교](#6.-옵티마이저-수렴-속도-비교)
7. [정리](#7.-정리)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

print("TensorFlow 버전:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

## 1. 수학적 기초

### SGD (Stochastic Gradient Descent)

$$\theta \leftarrow \theta - \eta \nabla L(\theta)$$

- $\theta$: 모델 파라미터
- $\eta$: 학습률 (learning rate)
- $\nabla L(\theta)$: 손실에 대한 그래디언트

### SGD with Momentum

$$v_t = \gamma v_{t-1} + \eta \nabla L(\theta)$$
$$\theta \leftarrow \theta - v_t$$

- $v_t$: 속도 벡터 (이전 그래디언트 정보 누적)
- $\gamma$: 모멘텀 계수 (보통 0.9)

### Adam (Adaptive Moment Estimation)

**1차 모멘트 (그래디언트 평균):**
$$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$$

**2차 모멘트 (그래디언트 제곱 평균):**
$$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$$

**편향 보정 (초기 0으로 초기화된 편향 수정):**
$$\hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$

**파라미터 업데이트:**
$$\theta \leftarrow \theta - \frac{\eta \hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

- 기본값: $\beta_1 = 0.9$, $\beta_2 = 0.999$, $\epsilon = 10^{-7}$
- 각 파라미터마다 적응적 학습률을 사용 → 빠른 수렴

## 2. 옵티마이저 생성 및 기본 사용법

In [ ]:
# ---------------------------------------------------
# 주요 옵티마이저 생성
# ---------------------------------------------------

# SGD: 가장 기본적인 옵티마이저
sgd = tf.keras.optimizers.SGD(
    learning_rate=0.01,
    momentum=0.9,       # 모멘텀 적용 (기본값 0.0)
    nesterov=True       # Nesterov 모멘텀 사용 여부
)

# Adam: 가장 널리 사용되는 옵티마이저
adam = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,         # 1차 모멘트 감쇠율
    beta_2=0.999,       # 2차 모멘트 감쇠율
    epsilon=1e-7        # 수치 안정성을 위한 작은 값
)

# RMSprop: 순환 신경망에 자주 사용
rmsprop = tf.keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,            # 이동 평균 감쇠율
    momentum=0.0,
    epsilon=1e-7
)

# AdamW: Adam + Weight Decay (L2 정규화와 다름, 디커플된 가중치 감쇠)
adamw = tf.keras.optimizers.AdamW(
    learning_rate=0.001,
    weight_decay=0.004  # 가중치 감쇠 계수
)

print("생성된 옵티마이저:")
for opt in [sgd, adam, rmsprop, adamw]:
    print(f"  {opt.__class__.__name__}: lr={opt.learning_rate.numpy():.4f}")

# ---------------------------------------------------
# 수동 그래디언트 적용 예시
# ---------------------------------------------------
print("\n=== 수동 그래디언트 적용 ===")

# 간단한 변수
w = tf.Variable(3.0, name='w')
b = tf.Variable(0.0, name='b')

# 목표: w=2.0, b=1.0 찾기
with tf.GradientTape() as tape:
    y_pred = w * 2.0 + b
    loss = (y_pred - 5.0) ** 2  # 목표값 5.0 (= 2*2 + 1)

gradients = tape.gradient(loss, [w, b])
adam.apply_gradients(zip(gradients, [w, b]))

print(f"업데이트 전: w=3.0, b=0.0")
print(f"업데이트 후: w={w.numpy():.4f}, b={b.numpy():.4f}")
print(f"그래디언트: dw={gradients[0].numpy():.4f}, db={gradients[1].numpy():.4f}")

## 3. 학습률의 영향

학습률은 모델 학습에서 가장 중요한 하이퍼파라미터 중 하나이다.
- **너무 큰 학습률**: 발산 (loss가 폭발적으로 증가)
- **너무 작은 학습률**: 수렴 속도가 느림 (학습 시간 증가)
- **적절한 학습률**: 빠르고 안정적인 수렴

In [ ]:
# ---------------------------------------------------
# 학습률 크기에 따른 학습 곡선 비교
# ---------------------------------------------------

def train_with_lr(learning_rate, epochs=50):
    """주어진 학습률로 간단한 회귀 문제 학습"""
    # 간단한 데이터 생성: y = 2x + 1
    np.random.seed(42)
    X = np.random.randn(100, 1).astype(np.float32)
    y = 2 * X + 1 + 0.1 * np.random.randn(100, 1).astype(np.float32)
    
    # 간단한 선형 모델
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, input_shape=(1,))
    ])
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
        loss='mse'
    )
    
    history = model.fit(X, y, epochs=epochs, verbose=0, batch_size=32)
    return history.history['loss']

# 세 가지 학습률 비교
lr_configs = {
    'lr=0.5 (너무 큼)': 0.5,
    'lr=0.01 (적절)': 0.01,
    'lr=0.0001 (너무 작음)': 0.0001,
}

plt.figure(figsize=(12, 5))

for label, lr in lr_configs.items():
    losses = train_with_lr(lr)
    # NaN이나 inf 방지를 위해 클리핑
    losses = [min(l, 50) if not np.isnan(l) else 50 for l in losses]
    plt.plot(losses, label=label, linewidth=2)

plt.xlabel('에포크')
plt.ylabel('MSE 손실')
plt.title('학습률 크기에 따른 수렴 비교 (SGD)')
plt.legend()
plt.ylim(0, 20)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("학습률 0.5: 발산 가능성 높음")
print("학습률 0.01: 빠르고 안정적인 수렴")
print("학습률 0.0001: 수렴하지만 매우 느림")

## 4. Learning Rate Scheduler

학습 초기에는 큰 학습률로 빠르게 진행하고, 후반부에는 작은 학습률로 세밀하게 수렴시키는 전략이다.

In [ ]:
# ---------------------------------------------------
# Learning Rate Scheduler 종류와 시각화
# ---------------------------------------------------

steps = np.arange(0, 1000)

# 1. ExponentialDecay: lr = initial_lr * decay_rate ^ (step / decay_steps)
exp_decay = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=200,    # 200 스텝마다 감쇠
    decay_rate=0.5,     # 50%씩 감소
    staircase=False     # 연속적 감쇠 (True면 계단형)
)

# 2. CosineDecay: 코사인 함수 형태로 학습률 감소
cosine_decay = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    alpha=0.0           # 최소 학습률 비율 (0 = 완전히 0까지 감소)
)

# 3. CosineDecayRestarts: 주기적으로 학습률을 리셋 (웜 리스타트)
cosine_restarts = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=0.1,
    first_decay_steps=200,  # 첫 번째 사이클 길이
    t_mul=2.0,              # 각 사이클마다 길이 2배
    m_mul=0.9               # 각 사이클마다 초기 lr 90%
)

# 학습률 값 계산
exp_lrs = [exp_decay(s).numpy() for s in steps]
cos_lrs = [cosine_decay(s).numpy() for s in steps]
cos_restart_lrs = [cosine_restarts(s).numpy() for s in steps]

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(steps, exp_lrs, color='blue', linewidth=2)
axes[0].set_title('ExponentialDecay')
axes[0].set_xlabel('학습 스텝')
axes[0].set_ylabel('학습률')
axes[0].grid(True, alpha=0.3)

axes[1].plot(steps, cos_lrs, color='green', linewidth=2)
axes[1].set_title('CosineDecay')
axes[1].set_xlabel('학습 스텝')
axes[1].set_ylabel('학습률')
axes[1].grid(True, alpha=0.3)

axes[2].plot(steps, cos_restart_lrs, color='red', linewidth=2)
axes[2].set_title('CosineDecayRestarts (Warm Restarts)')
axes[2].set_xlabel('학습 스텝')
axes[2].set_ylabel('학습률')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 옵티마이저에 스케줄러 적용 예시
print("=== 옵티마이저에 스케줄러 적용 ===")
print("""
# 스케줄러를 옵티마이저의 learning_rate에 직접 전달
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.96
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')
""")

## 5. ReduceLROnPlateau 콜백

검증 손실이 개선되지 않을 때 학습률을 자동으로 줄이는 콜백이다.

In [ ]:
# ---------------------------------------------------
# ReduceLROnPlateau 콜백 예시
# ---------------------------------------------------

# 간단한 MNIST 모델로 테스트
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(-1, 784).astype('float32') / 255.0
X_test  = X_test.reshape(-1, 784).astype('float32') / 255.0

# 빠른 실험을 위해 일부 데이터만 사용
X_small = X_train[:5000]
y_small = y_train[:5000]

def build_model():
    """간단한 MLP 분류 모델 생성"""
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),  # 의도적으로 큰 lr
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ReduceLROnPlateau 콜백 설정
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   # 모니터할 지표
    factor=0.5,           # 학습률 감소 배율 (lr = lr * factor)
    patience=3,           # 개선 없이 기다릴 에포크 수
    min_lr=1e-6,          # 최소 학습률 하한선
    min_delta=0.001,      # 개선으로 인정할 최소 변화량
    verbose=1             # 학습률 변경 시 출력
)

history = model.fit(
    X_small, y_small,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[reduce_lr],
    verbose=0
)

# 학습률 변화 추적 (콜백 내부에서 기록)
print(f"\n최종 학습률: {model.optimizer.learning_rate.numpy():.6f}")

## 6. 옵티마이저 수렴 속도 비교

동일한 MNIST 모델에 SGD, Adam, RMSprop을 적용하고 수렴 속도를 비교한다.

In [ ]:
# ---------------------------------------------------
# SGD / Adam / RMSprop 수렴 속도 비교
# ---------------------------------------------------

EPOCHS = 15
BATCH_SIZE = 64

# 각 옵티마이저 설정
optimizers_config = {
    'SGD (lr=0.01)': tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    'Adam (lr=0.001)': tf.keras.optimizers.Adam(learning_rate=0.001),
    'RMSprop (lr=0.001)': tf.keras.optimizers.RMSprop(learning_rate=0.001),
}

histories = {}

for name, optimizer in optimizers_config.items():
    print(f"\n학습 중: {name}")
    
    # 동일한 가중치 초기화를 위해 시드 고정 후 모델 재생성
    tf.random.set_seed(42)
    model = build_model()
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    start_time = time.time()
    history = model.fit(
        X_small, y_small,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        verbose=0
    )
    elapsed = time.time() - start_time
    
    histories[name] = history.history
    final_val_acc = history.history['val_accuracy'][-1]
    print(f"  완료 - 최종 검증 정확도: {final_val_acc:.4f}, 소요 시간: {elapsed:.1f}초")

# 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = ['blue', 'red', 'green']

for (name, history), color in zip(histories.items(), colors):
    epochs_range = range(1, EPOCHS + 1)
    
    axes[0].plot(epochs_range, history['loss'], label=name, color=color, linewidth=2)
    axes[1].plot(epochs_range, history['val_accuracy'], label=name, color=color, linewidth=2)

axes[0].set_xlabel('에포크')
axes[0].set_ylabel('훈련 손실')
axes[0].set_title('옵티마이저별 훈련 손실 비교')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('에포크')
axes[1].set_ylabel('검증 정확도')
axes[1].set_title('옵티마이저별 검증 정확도 비교')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 정리

### 옵티마이저 선택 가이드

| 옵티마이저 | 장점 | 단점 | 권장 사용 상황 |
|-----------|------|------|---------------|
| SGD + Momentum | 일반화 성능 좋음 | 학습률 튜닝 필요 | 이미지 분류, 최종 성능 중요 시 |
| Adam | 빠른 수렴, 학습률 튜닝 덜 필요 | 일부 태스크에서 과적합 가능 | 대부분의 경우 기본 선택 |
| RMSprop | 비정상 데이터에 강건 | Adam보다 수렴 느릴 수 있음 | 순환 신경망(RNN) |
| AdamW | Adam + 더 나은 정규화 | 추가 하이퍼파라미터 | Transformer 기반 모델 |

### Learning Rate Scheduler 선택

| 스케줄러 | 특징 | 권장 사용 상황 |
|---------|------|---------------|
| ExponentialDecay | 단순하고 예측 가능 | 일반적인 경우 |
| CosineDecay | 부드러운 감소 | 대부분의 딥러닝 태스크 |
| CosineDecayRestarts | 주기적 리셋으로 탈출 효과 | 로컬 미니멈 탈출 필요 시 |
| ReduceLROnPlateau | 자동 적응형 | 검증 손실 정체 시 자동 조정 필요 |

### 핵심 정리
- 학습률은 가장 중요한 하이퍼파라미터이다. Adam의 기본값 `0.001`이 좋은 출발점이다
- Adam은 대부분의 경우 좋은 성능을 보이지만, SGD+Momentum이 최종 성능에서 유리할 수 있다
- Learning Rate Scheduler를 사용하면 추가 비용 없이 성능을 향상시킬 수 있다